**MMSEQS2**

mmseqs/bin/mmseqs easy-search impairment_of_the_mitochondrial_integrity_and_biogenesis_genes.fasta realmergedprots_noen.faa mitimpair_noen.m8 -c 0.2 tmp --min-seq-id 0.3 --alt-ali 25

In [1]:
import pandas as pd
columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", 
           "qstart", "qend", "sstart", "send", "evalue", "bitscore"]

df = pd.read_csv('batstring.m8', sep='\t', names=columns)
print(df.head())

                     qseqid            sseqid  pident  length  mismatch  \
0  59463.ENSMLUP00000016363  mlXP_023610807.1   0.961     389        15   
1  59463.ENSMLUP00000016363  mmXP_036164646.1   0.953     387        18   
2  59463.ENSMLUP00000016363  mdXP_059550832.1   0.948     387        20   
3  59463.ENSMLUP00000016363  pkXP_036265305.1   0.935     387        25   
4  59463.ENSMLUP00000016363  moXP_036135537.1   0.912     387        34   

   gapopen  qstart  qend  sstart  send         evalue  bitscore  
0        0       1   387       1   389  3.536000e-254       780  
1        0       1   387       1   387  1.551000e-252       775  
2        0       1   387       1   387  9.320000e-251       770  
3        0       1   387       1   386  1.339000e-244       752  
4        0       1   387       1   387  1.546000e-239       738  


In [2]:
best_matches = pd.DataFrame()

bat_prefixes = ['rf', 'dr', 'ra', 'pv', 'pg', 'pa', 'pk', 'ph', 'pd', 'mm', 'ml', 'mb', 'mo', 'md', 'sb', 'ef', 'aj', 'pn', 'en']

for prefix in bat_prefixes:

    bat_matches = df[df['sseqid'].str.startswith(prefix)]

    best_bat_matches = bat_matches.loc[bat_matches.groupby('qseqid')['pident'].idxmax()]

    best_matches = pd.concat([best_matches, best_bat_matches], ignore_index=True)


print(best_matches)

                       qseqid            sseqid  pident  length  mismatch  \
0    59463.ENSMLUP00000000083  rfXP_032946134.1   0.964     284        10   
1    59463.ENSMLUP00000001427  rfXP_032943217.1   0.931     189        13   
2    59463.ENSMLUP00000001789  rfXP_032971332.1   0.959     524        21   
3    59463.ENSMLUP00000002616  rfXP_032990906.1   0.929     427        30   
4    59463.ENSMLUP00000002847  rfXP_032969289.1   0.807     209        39   
..                        ...               ...     ...     ...       ...   
488  59463.ENSMLUP00000014213    pnCAK6441801.1   0.995     660         3   
489  59463.ENSMLUP00000016071    pnCAK6440610.1   0.835     462        75   
490  59463.ENSMLUP00000016363    pnCAK6444154.1   0.975     248         6   
491  59463.ENSMLUP00000016529    pnCAK6437761.1   0.912     663        58   
492  59463.ENSMLUP00000022559    pnCAK6446407.1   0.980     157         3   

     gapopen  qstart  qend  sstart  send         evalue  bitscore  
0      

In [3]:
best_matches.to_csv('batstring_bestmatches.txt', sep='\t', index=False)

In [4]:
best_matches['bat_prefix'] = best_matches['sseqid'].str[:2]

# Group by 'qseqid' and count unique 'bat_prefix' values for each human protein
bat_counts_per_human = best_matches.groupby('qseqid')['bat_prefix'].nunique()

# Display the counts
print(bat_counts_per_human)

qseqid
59463.ENSMLUP00000000083    17
59463.ENSMLUP00000001427    17
59463.ENSMLUP00000001789    17
59463.ENSMLUP00000002616    17
59463.ENSMLUP00000002847    17
59463.ENSMLUP00000003315    17
59463.ENSMLUP00000003322    17
59463.ENSMLUP00000004009    17
59463.ENSMLUP00000004202    17
59463.ENSMLUP00000005054    17
59463.ENSMLUP00000005277    17
59463.ENSMLUP00000005366    17
59463.ENSMLUP00000005595    17
59463.ENSMLUP00000008027    17
59463.ENSMLUP00000009180    17
59463.ENSMLUP00000009404    17
59463.ENSMLUP00000010380    17
59463.ENSMLUP00000011160    17
59463.ENSMLUP00000011196    17
59463.ENSMLUP00000011227    17
59463.ENSMLUP00000011717    17
59463.ENSMLUP00000012090    17
59463.ENSMLUP00000013453    17
59463.ENSMLUP00000013787    17
59463.ENSMLUP00000014213    17
59463.ENSMLUP00000016071    17
59463.ENSMLUP00000016363    17
59463.ENSMLUP00000016529    17
59463.ENSMLUP00000022559    17
Name: bat_prefix, dtype: int64


In [5]:
qseqids_less_than = bat_counts_per_human[bat_counts_per_human < 17].index.tolist()
print("qseqids with less than 17 bat matches:", qseqids_less_than)
print(len(qseqids_less_than))

qseqids with less than 17 bat matches: []
0


In [6]:
best_matches['pident'].median()

0.933

PAIRED FASTA GENERATION

In [7]:
from Bio import SeqIO
import pandas as pd
import os


# File path
bat_fasta_file = "realmergedprots_noen.faa"  # FASTA file containing bat protein sequences
blast_results_file = "batstring_bestmatches.txt"  # Should contain the best_matches DataFrame
output_dir = "batstring_paired"  # Directory to store per-gene FASTA files

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to load sequences while handling duplicates
def load_fasta_with_duplicates(fasta_file):
    sequences = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        if record.id not in sequences:
            sequences[record.id] = record
        else:
            print(f"Duplicate ID found: {record.id}, ignoring this entry.")
    return sequences

# Load bat sequences
bat_sequences = load_fasta_with_duplicates(bat_fasta_file)

# Load best_matches DataFrame
blast_df = pd.read_csv(blast_results_file, sep="\t")

# Process each human gene and its associated bat genes
for qseqid, group in blast_df.groupby("qseqid"):
    output_file = os.path.join(output_dir, f"{qseqid}_bat_only.fasta")
    
    with open(output_file, "w") as output_fasta:
        # Write only aligned bat protein sequences
        for _, row in group.iterrows():
            sseqid_full = row["sseqid"]
            prefix = sseqid_full[:2]
            sseqid = sseqid_full

            if sseqid in bat_sequences:
                sseq = bat_sequences[sseqid].seq
                sdesc = bat_sequences[sseqid].description

                # Note the ID source based on prefix
                if prefix == "pn" or prefix == 'en':
                    id_source = "GenBank"
                else:
                    id_source = "RefSeq"

                # Write the bat protein sequence
                output_fasta.write(f">{sseqid} {sdesc} [{id_source}]\n{sseq}\n")
            else:
                print(f"SseqID {sseqid} not found in bat proteome.")

    print(f"Bat-only paired sequences for {qseqid} written to {output_file}")

print(f"All bat-only paired sequences written to directory: {output_dir}")


Bat-only paired sequences for 59463.ENSMLUP00000000083 written to batstring_paired\59463.ENSMLUP00000000083_bat_only.fasta
Bat-only paired sequences for 59463.ENSMLUP00000001427 written to batstring_paired\59463.ENSMLUP00000001427_bat_only.fasta
Bat-only paired sequences for 59463.ENSMLUP00000001789 written to batstring_paired\59463.ENSMLUP00000001789_bat_only.fasta
Bat-only paired sequences for 59463.ENSMLUP00000002616 written to batstring_paired\59463.ENSMLUP00000002616_bat_only.fasta
Bat-only paired sequences for 59463.ENSMLUP00000002847 written to batstring_paired\59463.ENSMLUP00000002847_bat_only.fasta
Bat-only paired sequences for 59463.ENSMLUP00000003315 written to batstring_paired\59463.ENSMLUP00000003315_bat_only.fasta
Bat-only paired sequences for 59463.ENSMLUP00000003322 written to batstring_paired\59463.ENSMLUP00000003322_bat_only.fasta
Bat-only paired sequences for 59463.ENSMLUP00000004009 written to batstring_paired\59463.ENSMLUP00000004009_bat_only.fasta
Bat-only paired 

In [11]:
from Bio import SeqIO
import pandas as pd
import os

# File paths
bat_fasta_file = "realmergedprots_filtered.faa"  # FASTA file containing bat protein sequences
blast_results_file = "housekeeping_bestmatches.txt"  # Should contain the best_matches DataFrame
output_dir = "housekeeping_paired"  # Directory to store per-gene FASTA files

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to load sequences while handling duplicates
def load_fasta_with_duplicates(fasta_file):
    sequences = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        if record.id not in sequences:
            sequences[record.id] = record
        else:
            print(f"Duplicate ID found: {record.id}, ignoring this entry.")
    return sequences

# Load bat sequences
bat_sequences = load_fasta_with_duplicates(bat_fasta_file)

# Load best_matches DataFrame
blast_df = pd.read_csv(blast_results_file, sep="\t")

# Process each human gene and its associated bat genes
for qseqid, group in blast_df.groupby("qseqid"):
    output_file = os.path.join(output_dir, f"{qseqid}_bat_only.fasta")
    
    with open(output_file, "w") as output_fasta:
        # Write only aligned bat protein sequences
        for _, row in group.iterrows():
            sseqid = row["sseqid"]  # Use sseqid directly as the key for lookup

            if sseqid in bat_sequences:
                sseq = bat_sequences[sseqid].seq
                sdesc = bat_sequences[sseqid].description

                # Write the bat protein sequence without altering the label
                output_fasta.write(f">{sseqid} {sdesc}\n{sseq}\n")
            else:
                print(f"SseqID {sseqid} not found in bat proteome.")

    print(f"Bat-only paired sequences for {qseqid} written to {output_file}")

print(f"All bat-only paired sequences written to directory: {output_dir}")

Bat-only paired sequences for P00558 written to housekeeping_paired\P00558_bat_only.fasta
Bat-only paired sequences for P04406 written to housekeeping_paired\P04406_bat_only.fasta
Bat-only paired sequences for P05388 written to housekeeping_paired\P05388_bat_only.fasta
Bat-only paired sequences for P60709 written to housekeeping_paired\P60709_bat_only.fasta
Bat-only paired sequences for P62277 written to housekeeping_paired\P62277_bat_only.fasta
All bat-only paired sequences written to directory: housekeeping_paired


Then MSA in command line (MAFFT) OPTIONAL

Nucleotide file generation

In [8]:
import os
from Bio import SeqIO

# File paths
nucleotide_fasta = "mergedprotscds.fasta"  # Large file with all nucleotide sequences
msa_input_dir = "batstring_paired"  # Directory with per-gene protein alignments
output_dir = "batstring_nucs"  # Directory to store codon alignments

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to parse IDs and handle repeated names
def extract_gene_id(header):
    """Extract the first gene ID from a header with duplicate IDs."""
    return header.split()[0]  # Take the first part of the header

# Load nucleotide sequences into a dictionary, using only the first part of the ID
print("Loading nucleotide sequences from mergedprotscds.fasta...")
nucleotide_dict = {}
for record in SeqIO.parse(nucleotide_fasta, "fasta"):
    gene_id = extract_gene_id(record.id)
    nucleotide_dict[gene_id] = record
print(f"Loaded {len(nucleotide_dict)} nucleotide sequences.")

# Process each MSA input file
for msa_file in os.listdir(msa_input_dir):
    if msa_file.endswith(".fasta") or msa_file.endswith(".fa"):
        msa_path = os.path.join(msa_input_dir, msa_file)
        gene_name = os.path.splitext(msa_file)[0]
        output_nucleotide_file = os.path.join(output_dir, f"{gene_name}_nucleotide.fasta")
        output_codon_alignment_file = os.path.join(output_dir, f"{gene_name}_codon_alignment.fasta")

        # Extract protein IDs from the MSA file
        protein_ids = []
        with open(msa_path, "r") as msa_handle:
            for record in SeqIO.parse(msa_handle, "fasta"):
                protein_id = extract_gene_id(record.id)  # Extract the first part of the header
                protein_ids.append(protein_id)
        
        # Match nucleotide sequences to the protein IDs
        matching_nucleotides = []
        for pid in protein_ids:
            if pid in nucleotide_dict:
                matching_nucleotides.append(nucleotide_dict[pid])
            else:
                print(f"Warning: Protein ID {pid} not found in nucleotide file.")
        
        # Write matching nucleotide sequences to a new file
        if matching_nucleotides:
            with open(output_nucleotide_file, "w") as output_handle:
                SeqIO.write(matching_nucleotides, output_handle, "fasta")
            print(f"Extracted nucleotide sequences for {gene_name}.")

            # Run pal2nal
            print(f"Running pal2nal for {gene_name}...")
            pal2nal_command = f"perl pal2nal.pl {msa_path} {output_nucleotide_file} -output fasta -verbose -nogap > {output_codon_alignment_file}"
            os.system(pal2nal_command)
            print(f"Codon alignment saved to {output_codon_alignment_file}.")
        else:
            print(f"No matching nucleotide sequences found for {gene_name}. Skipping...")

Loading nucleotide sequences from mergedprotscds.fasta...
Loaded 862310 nucleotide sequences.
Extracted nucleotide sequences for 59463.ENSMLUP00000000083_bat_only.
Running pal2nal for 59463.ENSMLUP00000000083_bat_only...
Codon alignment saved to batstring_nucs\59463.ENSMLUP00000000083_bat_only_codon_alignment.fasta.
Extracted nucleotide sequences for 59463.ENSMLUP00000001427_bat_only.
Running pal2nal for 59463.ENSMLUP00000001427_bat_only...
Codon alignment saved to batstring_nucs\59463.ENSMLUP00000001427_bat_only_codon_alignment.fasta.
Extracted nucleotide sequences for 59463.ENSMLUP00000001789_bat_only.
Running pal2nal for 59463.ENSMLUP00000001789_bat_only...
Codon alignment saved to batstring_nucs\59463.ENSMLUP00000001789_bat_only_codon_alignment.fasta.
Extracted nucleotide sequences for 59463.ENSMLUP00000002616_bat_only.
Running pal2nal for 59463.ENSMLUP00000002616_bat_only...
Codon alignment saved to batstring_nucs\59463.ENSMLUP00000002616_bat_only_codon_alignment.fasta.
Extracted 

Then MASCE then PAL2NAL

TREES GENERATION

In [9]:
from ete3 import Tree
from Bio import SeqIO
import os

# Paths
full_tree_file = "bat_tags_noen.nwk"  # Path to the full tree file
pal2nal_dir = "batstring_pal2nal"  # Directory with cleaned codon-aligned files
output_tree_dir = "batstring_trees"  # Directory to store pruned trees

# Species tag to full name mapping
tag_to_species = {
    "rf": "Rhinolophus_ferrumequinum",
    "dr": "Desmodus_rotundus",
    "ra": "Rousettus_aegyptiacus",
    "pv": "Pteropus_vampyrus",
    "pg": "Pteropus_giganteus",
    "pa": "Pteropus_alecto",
    "pk": "Pipistrellus_kuhlii",
    "ph": "Phyllostomus_hastatus",
    "pd": "Phyllostomus_discolor",
    "mm": "Myotis_myotis",
    "ml": "Myotis_lucifugus",
    "mb": "Myotis_brandtii",
    "mo": "Molossus_molossus",
    "md": "Myotis_daubentonii",
    "sb": "Saccopteryx_bilineata",
    "aj": "Artibeus_jamaicensis",
    "pn": "Pipistrellus_nathusii",
    "en": "Eptesicus_nilssonii",
}

# Create output directory if it doesn't exist
os.makedirs(output_tree_dir, exist_ok=True)

# Load the full tree
full_tree = Tree(full_tree_file, format=1)

# Process each alignment file
for alignment_file in os.listdir(pal2nal_dir):
    if alignment_file.endswith(".fasta"):
        alignment_path = os.path.join(pal2nal_dir, alignment_file)

        # Check if the file is empty
        if os.stat(alignment_path).st_size == 0:
            print(f"Skipping empty file: {alignment_file}")
            continue

        print(f"\nProcessing: {alignment_file}")

        # Extract sequence names from the alignment
        sequence_names = {}
        for record in SeqIO.parse(alignment_path, "fasta"):
            tag = record.id[:2]  # Extract the first two letters of the record ID
            if tag in tag_to_species:
                species_name = tag_to_species[tag]
                sequence_names[species_name] = record.id
            else:
                print(f"Warning: Unrecognized tag '{tag}' in {record.id}")

        # Debugging: Print extracted sequence names
        print(f"Extracted sequence names for {alignment_file}:")
        for species, seq_id in sequence_names.items():
            print(f"  {species}: {seq_id}")

        # Copy the full tree for modification
        unique_tree = full_tree.copy()

        # Rename leaves to match sequence names and remove unmatched leaves
        unmatched_leaves = []
        for leaf in unique_tree.get_leaves():
            if leaf.name in sequence_names:
                leaf.name = sequence_names[leaf.name]  # Rename leaf to match sequence ID
            else:
                unmatched_leaves.append(leaf.name)

        # Debugging: Print unmatched leaves
        if unmatched_leaves:
            print(f"Unmatched leaves for {alignment_file}: {unmatched_leaves}")

        # Remove unmatched leaves
        for leaf_name in unmatched_leaves:
            leaf = unique_tree.search_nodes(name=leaf_name)[0]
            leaf.detach()

        # Save the unique tree
        output_tree_path = os.path.join(output_tree_dir, f"{alignment_file}_pruned.nwk")
        unique_tree.write(outfile=output_tree_path, format=1)
        print(f"Unique tree saved to: {output_tree_path}")


Processing: 59463.ENSMLUP00000000083_bat_only_nucleotide_codon_aligned.fasta
Extracted sequence names for 59463.ENSMLUP00000000083_bat_only_nucleotide_codon_aligned.fasta:
  Rhinolophus_ferrumequinum: rfXP_032946134.1
  Pipistrellus_nathusii: pnCAK6438137.1
  Pipistrellus_kuhlii: pkXP_036306523.1
  Molossus_molossus: moXP_036123910.1
  Myotis_daubentonii: mdXP_059526062.1
  Myotis_myotis: mmXP_036195761.1
  Saccopteryx_bilineata: sbXP_066093091.1
  Desmodus_rotundus: drXP_053766493.1
  Artibeus_jamaicensis: ajXP_036983857.1
  Phyllostomus_hastatus: phXP_045691911.1
  Phyllostomus_discolor: pdXP_028375016.1
  Rousettus_aegyptiacus: raXP_015999162.1
  Pteropus_giganteus: pgXP_039715138.1
  Pteropus_alecto: paXP_006912343.1
  Pteropus_vampyrus: pvXP_011366803.1
  Myotis_lucifugus: mlXP_023612723.1
  Myotis_brandtii: mbXP_005875843.1
Unique tree saved to: batstring_trees\59463.ENSMLUP00000000083_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk

Processing: 59463.ENSMLUP00000001427_bat_o

In [10]:
#!/usr/bin/env python3

import os
import re

SOURCE_DIR = 'batstring_trees'
DEST_DIR = 'batstring_trees_labeled'

os.makedirs(DEST_DIR, exist_ok=True)

# Prefixes that should be labeled as {Foreground}
foreground_prefixes = ['mm', 'ml', 'md', 'mb', 'dr', 'rf']

# Regex: capture (1) node/tip name, (2) colon+branch_length
pattern = re.compile(r'([^\(\),:]+)(:[0-9\.eE\-\+]+)')

def replacer(match):
    """Decide {Foreground} vs. {Background}."""
    taxon_name = match.group(1)
    branch_length = match.group(2)
    if any(taxon_name.startswith(pref) for pref in foreground_prefixes):
        return f"{taxon_name}{{Foreground}}{branch_length}"
    else:
        return f"{taxon_name}{{Background}}{branch_length}"

for filename in os.listdir(SOURCE_DIR):
    if filename.endswith('.nwk'):
        source_file = os.path.join(SOURCE_DIR, filename)
        dest_file = os.path.join(DEST_DIR, filename)

        with open(source_file, 'r') as infile:
            data = infile.read()

        # --- FORCE ONE LINE ---
        # Remove all linebreaks:
        data = data.replace('\n', '').replace('\r', '')
        # (Optional) remove extra spaces if you like:
        # data = ' '.join(data.split())

        # Perform the labeling
        data_labeled = pattern.sub(replacer, data)

        # Write out WITHOUT adding an extra newline at the end
        with open(dest_file, 'w') as outfile:
            outfile.write(data_labeled)

        print(f"Processed {filename} -> {dest_file}")

Processed 59463.ENSMLUP00000000083_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk -> batstring_trees_labeled\59463.ENSMLUP00000000083_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk
Processed 59463.ENSMLUP00000001427_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk -> batstring_trees_labeled\59463.ENSMLUP00000001427_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk
Processed 59463.ENSMLUP00000001789_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk -> batstring_trees_labeled\59463.ENSMLUP00000001789_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk
Processed 59463.ENSMLUP00000002616_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk -> batstring_trees_labeled\59463.ENSMLUP00000002616_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk
Processed 59463.ENSMLUP00000002847_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk -> batstring_trees_labeled\59463.ENSMLUP00000002847_bat_only_nucleotide_codon_aligned.fasta_pruned.nwk
Processed 59463.ENSMLUP00000003315_bat_only_nucleotide_

DN DS

In [12]:
import os
import re

# Define the directory containing your .log files
log_dir = 'batstring_busted'  # Update this if your directory name is different

# Output file to store the protein names
output_file = 'batstring_bustede1_prots_nofdr.txt'

# Open the output file in write mode
with open(output_file, 'w') as out_f:

    # Iterate over all .log files in the directory
    for filename in os.listdir(log_dir):
        if filename.endswith('.log'):
            filepath = os.path.join(log_dir, filename)
            # Extract the protein name from the filename based on the rules
            # Split the filename by '_'
            parts = filename.split('_')
            if parts[0].startswith('NP') or parts[0].startswith('XP'):
                # If the protein name starts with 'NP' or 'XP', join the first two parts
                protein_name = '_'.join(parts[:2])
            else:
                # Otherwise, the protein name is the first part
                protein_name = parts[0]

            # Open and read the log file
            with open(filepath, 'r') as log_f:
                lines = log_f.readlines()
                for line in lines:
                    if 'Likelihood ratio test for episodic diversifying positive selection' in line:
                        # Extract the p-value from this line
                        match = re.search(r'p =\s*([\d\.eE+-]+)', line)
                        if match:
                            p_value_str = match.group(1)
                            try:
                                p_value = float(p_value_str)
                                if p_value < 0.05:
                                    # Write the protein name to the output file
                                    out_f.write(protein_name + '\n')
                                    print(f"{protein_name}: p-value {p_value} < 0.05, added to {output_file}")
                                else:
                                    print(f"{protein_name}: p-value {p_value} >= 0.05, not added")
                            except ValueError:
                                print(f"{protein_name}: Could not convert p-value '{p_value_str}' to float")
                        else:
                            print(f"{protein_name}: p-value not found in line: {line.strip()}")
                        # Break after processing the line
                        break
                else:
                    # If we didn't find the line with 'Likelihood ratio test...'
                    print(f"{protein_name}: 'Likelihood ratio test' line not found in {filename}")


59463.ENSMLUP00000000083: p-value 0.5 >= 0.05, not added
59463.ENSMLUP00000001427: p-value 0.5 >= 0.05, not added
59463.ENSMLUP00000001789: p-value 0.5 >= 0.05, not added
59463.ENSMLUP00000002616: p-value 0.5 >= 0.05, not added
59463.ENSMLUP00000002847: p-value 0.4593 >= 0.05, not added
59463.ENSMLUP00000003315: p-value 0.5 >= 0.05, not added
59463.ENSMLUP00000003322: p-value 0.5 >= 0.05, not added
59463.ENSMLUP00000004009: p-value 0.3741 >= 0.05, not added
59463.ENSMLUP00000004202: p-value 0.2994 >= 0.05, not added
59463.ENSMLUP00000005054: 'Likelihood ratio test' line not found in 59463.ENSMLUP00000005054_bat_only_nucleotide_codon_aligned.fasta_busted.log
59463.ENSMLUP00000005366: p-value 0.2931 >= 0.05, not added
59463.ENSMLUP00000005595: p-value 0.2989 >= 0.05, not added
59463.ENSMLUP00000008027: p-value 0.5 >= 0.05, not added
59463.ENSMLUP00000009180: p-value 0.5 >= 0.05, not added
59463.ENSMLUP00000009404: 'Likelihood ratio test' line not found in 59463.ENSMLUP00000009404_bat_onl

FDR

In [1]:
#!/usr/bin/env python3

import os
import re
from statsmodels.stats.multitest import multipletests

# Directory containing your .log files
log_dir = 'batecmreg_busted'

# Output file
output_file = 'batecmreg_bustede1_FDR.txt'

# We will store all (protein_name, p_value) in a list for later correction
pvals_data = []

# 1. Parse each .log file and extract p-values
for filename in os.listdir(log_dir):
    if filename.endswith('.log'):
        filepath = os.path.join(log_dir, filename)
        # Extract the protein name from the filename
        parts = filename.split('_')
        if parts[0].startswith('NP') or parts[0].startswith('XP'):
            protein_name = '_'.join(parts[:2])
        else:
            protein_name = parts[0]

        # Read the log file
        with open(filepath, 'r') as log_f:
            lines = log_f.readlines()
            found_pval = False
            for line in lines:
                if 'Likelihood ratio test for episodic diversifying positive selection' in line:
                    # Extract the p-value
                    match = re.search(r'p =\s*([\d\.eE+-]+)', line)
                    if match:
                        p_value_str = match.group(1)
                        try:
                            p_value = float(p_value_str)
                            # Store this (protein, p-value)
                            pvals_data.append((protein_name, p_value))
                            found_pval = True
                        except ValueError:
                            print(f"{protein_name}: Could not convert p-value '{p_value_str}' to float")
                    else:
                        print(f"{protein_name}: p-value not found in line: {line.strip()}")
                    break  # Stop after finding the line
            if not found_pval:
                print(f"{protein_name}: 'Likelihood ratio test' line not found or no p-value in {filename}")

# 2. Once we've collected all p-values, apply FDR correction
p_values = [x[1] for x in pvals_data]  # just the numeric p-values

if len(p_values) == 0:
    print("No p-values found. Exiting.")
    exit()

# Benjamini-Hochberg procedure (FDR)
# returns: reject (boolean array), pvals_corrected, alphacSidak, alphacBonf
reject_array, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

# 3. Write results to file (only those that pass FDR, i.e., q < 0.05)
with open(output_file, 'w') as out_f:
    for i, (protein_name, original_pval) in enumerate(pvals_data):
        fdr_pval = pvals_corrected[i]
        if reject_array[i]:
            # This means fdr_pval < 0.05
            out_f.write(protein_name + '\n')
            print(f"{protein_name}: raw p = {original_pval:.4g}, FDR p = {fdr_pval:.4g} (SIGNIFICANT)")
        else:
            print(f"{protein_name}: raw p = {original_pval:.4g}, FDR p = {fdr_pval:.4g} (not significant)")

NP_001159718.1: 'Likelihood ratio test' line not found or no p-value in NP_001159718.1_aligned_codon_alignment.fasta_busted.log
NP_036597.1: 'Likelihood ratio test' line not found or no p-value in NP_036597.1_aligned_codon_alignment.fasta_busted.log
NP_079279.3: 'Likelihood ratio test' line not found or no p-value in NP_079279.3_aligned_codon_alignment.fasta_busted.log
NP_000005.2: raw p = 0.4814, FDR p = 0.5 (not significant)
NP_000020.1: raw p = 0.2562, FDR p = 0.5 (not significant)
NP_000053.2: raw p = 0.1706, FDR p = 0.5 (not significant)
NP_000090.1: raw p = 0.5, FDR p = 0.5 (not significant)
NP_000091.1: raw p = 0.129, FDR p = 0.5 (not significant)
NP_000120.2: raw p = 0.5, FDR p = 0.5 (not significant)
NP_000122.1: raw p = 0.5, FDR p = 0.5 (not significant)
NP_000124.1: raw p = 0.5, FDR p = 0.5 (not significant)
NP_000176.2: raw p = 0.5, FDR p = 0.5 (not significant)
NP_000286.3: raw p = 0.0754, FDR p = 0.5 (not significant)
NP_000292.1: raw p = 0.5, FDR p = 0.5 (not significant

EXAMINATION OF FITMG94 and R part